<a href="https://colab.research.google.com/github/Snowdenstyll/keno/blob/main/jupyterNotebooks/Keno_Lottery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [58]:
df = pd.read_csv("content/2024.csv")

In [59]:
df.head()
df.tail()
df.info()
df.describe()
df.drop(['PlayDate', 'AP'], axis=1, inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   PlayDate  128 non-null    object
 1   AP        128 non-null    object
 2   N01       128 non-null    int64 
 3   N02       128 non-null    int64 
 4   N03       128 non-null    int64 
 5   N04       128 non-null    int64 
 6   N05       128 non-null    int64 
 7   N06       128 non-null    int64 
 8   N07       128 non-null    int64 
 9   N08       128 non-null    int64 
 10  N09       128 non-null    int64 
 11  N10       128 non-null    int64 
 12  N11       128 non-null    int64 
 13  N12       128 non-null    int64 
 14  N13       128 non-null    int64 
 15  N14       128 non-null    int64 
 16  N15       128 non-null    int64 
 17  N16       128 non-null    int64 
 18  N17       128 non-null    int64 
 19  N18       128 non-null    int64 
 20  N19       128 non-null    int64 
 21  N20       128 no

In [60]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

In [61]:
# All our games
number_of_rows = df.values.shape[0]
# Amount of games we need to take into consideration for prediction
window_length = 3
number_of_features = df.values.shape[1]

X = np.empty([ number_of_rows - window_length, window_length, number_of_features], dtype=float)
y = np.empty([ number_of_rows - window_length, number_of_features], dtype=float)


In [62]:
for i in range(0, number_of_rows-window_length):
    X[i] = transformed_df.iloc[i : i+window_length, 0 : number_of_features]
    y[i] = transformed_df.iloc[i+window_length : i+window_length+1, 0 : number_of_features]

In [63]:
# Recurrent Neural Netowrk (RNN) with Long Short Term Memory (LSTM)
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional, Dropout
batch_size = 30

# Initialising the RNN
model = Sequential()
# Adding the input layer and the LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a first Dropout layer
model.add(Dropout(0.2))
# Adding a second LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a second Dropout layer
model.add(Dropout(0.2))
# Adding a third LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = True)))
# Adding a fourth LSTM layer
model.add(Bidirectional(LSTM(256,
                        input_shape = (window_length, number_of_features),
                        return_sequences = False)))
# Adding a fourth Dropout layer
model.add(Dropout(0.2))
# Adding the first output layer
model.add(Dense(70))
# Adding the last output layer
model.add(Dense(number_of_features))

In [64]:
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

model.compile(optimizer=Adam(learning_rate=0.0001), loss ='mse', metrics=['accuracy'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=90)

# Train the model
history = model.fit(X, y, batch_size=batch_size, epochs=1000, verbose=2)
#history = model.fit(X_train, y_train, batch_size=100, epochs=600, verbose=2)

Epoch 1/1000
5/5 - 15s - loss: 0.9949 - accuracy: 0.0400 - 15s/epoch - 3s/step
Epoch 2/1000
5/5 - 0s - loss: 0.9937 - accuracy: 0.0560 - 66ms/epoch - 13ms/step
Epoch 3/1000
5/5 - 0s - loss: 0.9925 - accuracy: 0.0800 - 66ms/epoch - 13ms/step
Epoch 4/1000
5/5 - 0s - loss: 0.9915 - accuracy: 0.1040 - 68ms/epoch - 14ms/step
Epoch 5/1000
5/5 - 0s - loss: 0.9905 - accuracy: 0.0800 - 63ms/epoch - 13ms/step
Epoch 6/1000
5/5 - 0s - loss: 0.9896 - accuracy: 0.0560 - 67ms/epoch - 13ms/step
Epoch 7/1000
5/5 - 0s - loss: 0.9879 - accuracy: 0.0800 - 65ms/epoch - 13ms/step
Epoch 8/1000
5/5 - 0s - loss: 0.9867 - accuracy: 0.0960 - 70ms/epoch - 14ms/step
Epoch 9/1000
5/5 - 0s - loss: 0.9853 - accuracy: 0.1040 - 64ms/epoch - 13ms/step
Epoch 10/1000
5/5 - 0s - loss: 0.9824 - accuracy: 0.0960 - 64ms/epoch - 13ms/step
Epoch 11/1000
5/5 - 0s - loss: 0.9801 - accuracy: 0.0880 - 67ms/epoch - 13ms/step
Epoch 12/1000
5/5 - 0s - loss: 0.9773 - accuracy: 0.0800 - 64ms/epoch - 13ms/step
Epoch 13/1000
5/5 - 0s - lo

In [65]:
loss, accuracy = model.evaluate(X, y, verbose=1)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

4/4 [==============================] - 2s 9ms/step - loss: 0.0202 - accuracy: 0.7920
Test Loss: 0.020217876881361008, Test Accuracy: 0.7919999957084656


In [66]:
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)
#history = model.fit(X_train, y_train, batch_size=50, epochs=100, verbose=2)

In [88]:
df1 = df.copy()
to_predict = df1.tail(window_length)
to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)
to_predict

<ipython-input-88-efb52fef0daf>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  to_predict.drop([to_predict.index[-1]],axis=0, inplace=True)


,N01,N02,N03,N04,N05,N06,N07,N08,N09,N10,N11,N12,N13,N14,N15,N16,N17,N18,N19,N20
125,1,9,10,19,20,22,24,26,28,31,34,37,38,42,45,55,60,61,67,70
126,2,3,4,8,10,14,18,20,30,31,33,44,45,48,57,59,60,61,68,70


In [108]:
prediction = np.array(df.tail(1))

x_next = scaler.transform(to_predict)
y_pred = model.predict(np.array([x_next]))

temp = np.array(scaler.inverse_transform(y_pred).astype(int)[0])
combined_array = np.concatenate((temp, np.array(prediction[0])))
winning_numbers = np.bincount(combined_array)
winners = np.unique(combined_array)

uniques, uniq_idx, counts = np.unique(combined_array,return_index=True,return_counts=True)
duplicates = combined_array[ uniq_idx[counts>=2] ]  # <--- Get duplicates

print("The predicted numbers for the lottery game are:", scaler.inverse_transform(y_pred).astype(int)[0])
print("The actual numbers in the last lottery game were:", prediction[0])
print("\n")
print("winners:", duplicates)
print(len(duplicates)/20)


1/1 [==============================] - 0s 20ms/step
The predicted numbers for the lottery game are: [ 4  9 13 16 18 21 24 27 32 35 37 42 44 47 50 54 56 60 63 67]
The actual numbers in the last lottery game were: [ 6 23 28 29 31 32 36 37 39 40 44 49 51 52 54 60 61 62 63 68]


winners: [32 37 44 54 60 63]
0.3


# Predict the next set of numbers

In [109]:
next = df1.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
y_next_pred = model.predict(np.array([x_next]))
prediction = scaler.inverse_transform(y_next_pred).astype(int)
print("The predicted numbers for the lottery game:", temp)
print("\n")
top_5_indices = np.argsort(y_next_pred[0], axis=0)[-5:]
top_5_numbers = []
for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]
    # Append the number to the top_5_numbers array
    top_5_numbers.append(number)


print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)


1/1 [==============================] - 0s 20ms/step
The predicted numbers for the lottery game: [ 4  9 13 16 18 21 24 27 32 35 37 42 44 47 50 54 56 60 63 67]


The top 5 indices [15 17 11 13 12]
The top 5 numbers: [55, 61, 42, 49, 46]


In [110]:
next = df1.copy()
next = next.tail(window_length)
next = np.array(next)

x_next = scaler.transform(next)
to_predict = np.array(to_predict)
scaled_to_predict = scaler.transform(to_predict)

# Assuming y_pred is the output of your model predictions
y_pred = model.predict(np.array([scaled_to_predict]))

# Get the actual values of the top 10 predictions
prediction = scaler.inverse_transform(y_pred).astype(int)

# Get the indices of the top 10 predictions (change axis=0 if predictions are along a different axis)
top_10_indices = np.argsort(y_pred[0], axis=0)[-10:]
top_5_indices = np.argsort(y_pred[0], axis=0)[-5:]

top_10_numbers = []
top_5_numbers = []

# index [0,2,4] usually are correct
# Loop through each index in top_10_indices
for index in top_10_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_10_numbers.append(number)

for index in top_5_indices:
    # Access the corresponding number in the top_10_predictions array
    number = prediction[0, index]

    # Append the number to the top_10_numbers array
    top_5_numbers.append(number)

print("The top 5 indices", top_5_indices)
print("The top 5 numbers:", top_5_numbers)
print("\n")

print("The top 10 indices", top_10_indices)
print("The top 10 numbers:", top_10_numbers)
print("\n")

print("The predicted numbers:", prediction)


1/1 [==============================] - 0s 21ms/step
The top 5 indices [8 3 1 2 0]
The top 5 numbers: [32, 16, 9, 13, 4]


The top 10 indices [ 5  6  9  4 11  8  3  1  2  0]
The top 10 numbers: [21, 24, 35, 18, 42, 32, 16, 9, 13, 4]


The predicted numbers: [[ 4  9 13 16 18 21 24 27 32 35 37 42 44 47 50 54 56 60 63 67]]
